In [0]:
display(dbutils.fs.ls('/Volumes/workspace/alura/c3'))

path = '/Volumes/workspace/alura/c3'

import os
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import plotly.express as px

%pip install Jinja2

In [0]:
cursos_raw = spark.read\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .format('csv')\
    .load(os.path.join(path,'cursos-prouni.csv'))

reajuste_raw = spark.read\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .format('csv')\
    .load(os.path.join(path, 'reajuste.csv'))

In [0]:
display(cursos_raw.limit(5))

In [0]:
display(reajuste_raw.limit(5))

In [0]:
cursos = cursos_raw.drop('curso_busca')
cursos = cursos.withColumnRenamed('nome', 'nome_curso')
cursos = cursos.dropna(how="any")

reajuste = reajuste_raw

cursos\
    .select('nome_curso')\
    .filter(cursos.nome_curso == 'Medicina')\
    .agg(F.count('nome_curso'))\
    .show()

cursos\
    .select('nome_curso', 'turno')\
    .filter(cursos.nome_curso == 'Medicina')\
    .groupBy('turno')\
    .agg(F.count('nome_curso'))\
    .show()

cursos\
    .select('nome_curso', 'bolsa_integral_cotas')\
    .filter(cursos.nome_curso == 'Medicina')\
    .agg(F.sum('bolsa_integral_cotas'))\
    .show()

In [0]:
reajuste = reajuste.withColumn('constant', F.lit(1))
cursos = cursos.withColumn('constant', F.lit(1))

w_reajuste = Window.orderBy("constant")
w_cursos = Window.orderBy('constant')

cursos = cursos.withColumn("contador", F.row_number().over(w_cursos))
reajuste = reajuste.withColumn("contador", F.row_number().over(w_reajuste))

cursos = cursos.drop('constant')
reajuste = reajuste.drop('constant')

df = cursos.join(reajuste, on='contador', how='inner')
df = df.drop('contador')

In [0]:
display(df)

In [0]:
display(df)

In [0]:
colunas_bolsa = [c for c in df.columns if "bolsa" in c.lower()]

display(df
    .select(*colunas_bolsa)
    .filter(cursos.nome_curso == 'Medicina')
    .describe())

df_medicina = df.filter(df.nome_curso == 'Medicina')

In [0]:
display(df_medicina.orderBy('mensalidade', ascending=False))

In [0]:
agrupados = df_medicina\
    .select(['mensalidade', 'uf_busca'])\
    .filter((df_medicina.nome_curso == 'Medicina') &
            ((df_medicina.turno == 'Integral') | (df_medicina.turno == 'Matutino'))
            )\
    .groupBy('uf_busca')\
    .agg(F.mean('mensalidade').alias('mensalidade'))\
    .orderBy("mensalidade", ascending=False)

display(agrupados)

agrupados_pd = agrupados.toPandas()


fig = px.bar(
    agrupados_pd,
    x="uf_busca",
    y="mensalidade",
    title="Mensalidade média por UF (ordenada)",
    labels={"uf_busca": "UF", "mensalidade": "Mensalidade Média"}
)
fig.show()

In [0]:
agrupados = df\
    .select('bolsa_integral_cotas', 'uf_busca')\
    .filter(df.nome_curso == 'Medicina')\
    .groupBy('uf_busca')\
    .agg(F.sum('bolsa_integral_cotas').alias('cotas'))\
    .orderBy('cotas', ascending=False)\

display(agrupados)

agrupados_pd = agrupados.toPandas()

fig = px.bar(
    agrupados_pd,
    x="uf_busca",
    y="cotas",
    title="Total de Cotas por UF (ordenada)",
    labels={"uf_busca": "UF", "cotas": "Total Cotas"}
)
fig.show()

In [0]:
nomes_cidades = {
"Sao Luis": "Capital",
"Joao Pessoa":"Capital",
"Belem":"Capital",
"Itaperuna":"Município",
"Vitoria":"Município",
"Franca":"Município",
"Sao Paulo":"Capital",
"Guarapuava":"Município",
"Campo Mourao":"Município",
"Montes Claros":"Município",
"Rio Branco":"Município",
"Imperatriz":"Município",
"Vespasiano":"Município",
"Porto Velho": "Município"
}

dados = list(nomes_cidades.items())

class_cidades = spark.createDataFrame(dados, ["cidade", "classificacao_cidades"])

class_cidades = class_cidades.withColumnRenamed('cidade', 'cidade_busca')\
                .dropDuplicates(subset=['cidade_busca'])

df_medicina = df_medicina.join(class_cidades, on='cidade_busca', how='inner')

In [0]:
agrupados = df_medicina\
    .select('classificacao_cidades', 'bolsa_integral_cotas')\
    .groupBy('classificacao_cidades')\
    .agg(F.sum('bolsa_integral_cotas').alias('cotas'))\
    .orderBy('cotas', ascending=False)

display(agrupados)

agrupados_pd = agrupados.toPandas()

fig = px.bar(
    agrupados_pd,
    x="classificacao_cidades",
    y="cotas",
    title="Total de Cotas por Tipo de Cidade",
    labels={"cotas": "Cotas", "classificacao_cidades": "Classificação Cidades"}
)
fig.show()


In [0]:
aux = df_medicina.select('mensalidade','bolsa_integral_cotas','bolsa_integral_ampla','bolsa_parcial_cotas','bolsa_parcial_ampla','nota_integral_ampla', 'nota_integral_cotas', 'nota_parcial_ampla','nota_parcial_cotas')

aux.toPandas().corr().style.background_gradient(cmap="Reds",vmin=-1,vmax=1)

In [0]:
df_medicina.plot.scatter(x='mensalidade',y='bolsa_parcial_cotas', trendline='ols', trendline_color_override = 'red')


In [0]:
df_medicina.plot.scatter(x='nota_parcial_ampla',y='nota_parcial_cotas', trendline='ols', trendline_color_override = 'red')

In [0]:
df\
    .select('mensalidade')\
    .filter(df.nome_curso == 'Medicina')\
    .plot.box()

In [0]:
df\
    .select('mensalidade')\
    .filter(df.nome_curso == 'Enfermagem')\
    .plot.box()

In [0]:
nomes_universidades = (
    df_medicina
    .select('universidade_nome')
    .dropDuplicates(['universidade_nome'])
)

df_medicina = df_medicina\
    .withColumn('universidade_nome',
        F.when(df_medicina.universidade_nome == 'Centro Universitário Integrado de Campo Mourão', 
        'Centro Universitário Integrado de Campo Mourão - CUICM')
        .otherwise(df_medicina.universidade_nome)
    )

display(nomes_universidades)

In [0]:
display(
    df_medicina
    .select('universidade_nome')
    .dropDuplicates(['universidade_nome'])
    .filter(F.col('universidade_nome').contains('Universidade'))
)

In [0]:
separado = df_medicina\
    .select('universidade_nome')\
    .withColumn('universidade_list', F.split(df_medicina.universidade_nome, " - "))\
    .withColumn("sigla", F.col("universidade_list")[1])\
    .drop('universidade_list')\
    .dropDuplicates(['universidade_nome'])

display(separado)

In [0]:
df_medicina = df_medicina\
    .join(
        separado,
        on='universidade_nome',
        how='inner'
    )

display(df_medicina)

In [0]:
df_medicina\
    .select('sigla', 'nota_integral_ampla')\
    .groupBy('sigla')\
    .agg(F.mean('nota_integral_ampla').alias('media'))\
    .orderBy('media', ascending=False)\
    .plot.bar(x='sigla', y='media')